In [5]:
domain = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# Order: T W O F U R
D = [domain]*6
# Manually solve node consistency
D[3] = [1]
D[0] = [5, 6, 7, 8, 9]
# Defining  constraints
# Digonals:
def CDi(x, y):
    return True
# Default:
def CDe(x, y):
    return x != y
# C00: diagonal
# C01: default
def C02(T, O):
    return (T != O) and (O == (2*T)%10 or O == (2*T)%10 + 1)
# C03: default
# C04: default
# C05: default
# C10: default
# C11: diagonal
def C12(W, O):
    return (O != W) and ((O%2 == 1) and (2*W)/10 >= 1)
# C13: default
def C14(W, U):
    return (W != 0) and (U == (2*W)%10 or U == (2*W)%10 + 1)
# C15: default
def C20(O, T):
    return (T != 0) and (O == (2*T)%10 or O == (2*T)%10 + 1)
def C21(O, W):
    return (O != W) and ((O%2 == 1) and (2*W)/10 >= 1)
# C22: diagonal
# C23: default
def C24(O, U):
    return (O != U) and ((U%2 == 1) and (2*O)/10 >= 1)
def C25(O, R):
    return (O != R) and (R == (2*O)%10)
# C30: default
# C31: default
# C32: default
# C33: diagonal
# C34: default
# C35: default
# C40: default
def C41(U, W):
    return (W != 0) and (U == (2*W)%10 or U == (2*W)%10 + 1)
def C42(U, O):
    return (O != U) and ((U%2 == 1) and (2*O)/10 >= 1)
# C43: default
# C44: diagonal
# C45: default
def C52(R, O):
    return (O != R) and (R == (2*O)%10)
# rest are defaults or diagonals

# Consistency matrix
C = [[CDi, CDe, C02, CDe, CDe, CDe],
     [CDe, CDi, C12, CDe, C14, CDe],
     [C20, C21, CDi, CDe, C24, C25],
     [CDe, CDe, CDe, CDi, CDe, CDe],
     [CDe, C41, C42, CDe, CDi, CDe].
     [CDe, CDe, C52, CDe, CDe, CDi]]


2